In [ ]:
###############################################################################
# Create synthetic training data for the stochastic competitive Lotka–Volterra model
###############################################################################

import numpy as np
from dynamics import stoch_dyn_LVE

# Simulation parameters
dt = 0.01
num_trajectories = 10000    # number of Monte Carlo runs per initial condition
num_inits = 100             # number of unique initial conditions
T = 50                      # number of time steps per trajectory

# State dimension
nx = 2
nw = 2

# Define grid of initial conditions (x, y)
x0_vals = np.linspace(0.1, 1.2, int(np.sqrt(num_inits)))
y0_vals = np.linspace(0.1, 1.2, int(np.sqrt(num_inits)))
initial_conditions = np.array([[x, y] for x in x0_vals for y in y0_vals])

# Pre-allocate arrays
mean_initial = []
mean_final = []
cov_initial = []
cov_final = []

print("Generating synthetic LVE data...")

# Iterate over initial conditions
for ic in initial_conditions:
    x0, y0 = ic
    X = np.zeros((num_trajectories, nx))
    X_next = np.zeros((num_trajectories, nx))
    
    # Monte Carlo simulation
    for i in range(num_trajectories):
        xk, yk = x0, y0
        for t in range(T):
            xkw, ykw = np.random.randn(), np.random.randn()
            [xkp1ykp1, _, _] = stoch_dyn_LVE([xk, yk, xkw, ykw])
            xkp1, ykp1 = xkp1ykp1.flatten()
            if t == T - 2:
                X[i, :] = [xk, yk]
                X_next[i, :] = [xkp1, ykp1]
            xk, yk = xkp1, ykp1

    # Compute moments
    mu_i = np.mean(X, axis=0)
    mu_f = np.mean(X_next, axis=0)
    cov_i = np.cov(X.T)
    cov_f = np.cov(X_next.T)
    
    mean_initial.append(mu_i)
    mean_final.append(mu_f)
    cov_initial.append(cov_i)
    cov_final.append(cov_f)

# Convert to numpy arrays
mean_initial = np.array(mean_initial).reshape(num_inits, nx, 1)
mean_final = np.array(mean_final).reshape(num_inits, nx, 1)
cov_initial = np.array(cov_initial).reshape(num_inits, nx, nx)
cov_final = np.array(cov_final).reshape(num_inits, nx, nx)

In [ ]:
# Save to disk (consistent with SPINODE naming)
np.save("LVE/data/mean_initial.npy", mean_initial)
np.save("LVE/data/mean_final.npy", mean_final)
np.save("LVE/data/cov_initial.npy", cov_initial)
np.save("LVE/data/cov_final.npy", cov_final)

Synthetic LVE data saved in 'LVE/Data/'
